# theory

Прежде чем извлечь признаки из текста, упростим его.  

Рассмотрим этапы предобработки текста:  
* Токенизация (англ. tokenization) — разбиение текста на токены: отдельные фразы, слова, символы.  
* Лемматизация (англ. lemmatization) — приведение слова к начальной форме (лемме).  

Функция лемматизации русского текста есть в библиотеках:  
- pymorphy2 (англ. python morphology, «морфология для Python»),  
- UDPipe (англ. universal dependencies pipeline, «конвейер для построения общих зависимостей»),  
- pymystem3.  

https://habr.com/ru/post/503420/  

Чтобы машины воспринимали слова, картинки или аудио, их преобразовывают в векторный вид. Когда работают с текстом, его тоже переводят в векторный формат, или векторные представления. Частный случай этих представлений — **word embeddings** (англ. «слова-вложения»; «эмбеддинги»). Работают они так: сложная структура (текст) вкладывается в более простую — вектор.  

Векторы-эмбеддинги содержат данные о соотношении разных слов и их свойствах. Привычное понимание свойства слова, его смысла и контекста справедливо и для машинного обучения.  

Свойства — это скрытые смыслы слова. Допустим, «моряк» содержит смыслы: «мужчина», «профессия», «человек» и «морской». Смысл, или семантика слова, — это лексическое значение слова, его отличие от других слов. Но слово обычно не живёт само по себе, его окружают другие слова. Например, «лента» во фразе: «Красная лента в каштановых волосах» отличается от «ленты» из предложения: «Положите продукты на ленту перед кассой». Именно контекст и определяет смысл слов.  

Эти понятия пригодятся нам в определении близости слов в их векторном представлении. Так близкие, или похожие векторы, могут отображать похожие слова. Сходство векторов рассчитывается знакомым вам евклидовым расстоянием: чем меньше расстояние, тем сильнее похожи векторы.

# reqs

`conda activate sandbox`  

https://pypi.org/project/pymystem3/  
  
`pip install pymystem3`  

Один раз после установки _nltk_ надо скачать стоп-слова:

In [35]:
# import nltk
# nltk.download('stopwords')

(у меня в _sandbox_ это уже было сделано)

Для **RuBERT**'a:  

```
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
conda install -c huggingface transformers
```

https://huggingface.co/docs/transformers/index   

# init

In [42]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

import requests

In [7]:
PREFIX = 'https://code.s3.yandex.net' # PREFIX + 

# data

русскоязычный корпус коротких текстов **RuTweetCorp**: http://study.mokoron.com/  

Перед вами уменьшенный датасет — 5000 записей. Каждая запись содержит текст твита и оценку его тональности. Если пост позитивный, то метка «1», если негативный — «0». (Твиты могут содержать обсценную лексику)  

Выгрузить данные моджно в задании про sentiment analysis:

In [51]:
train = pd.read_csv('./files/tweets_lemm_train.csv')

train.head()

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


In [52]:
test = pd.read_csv('./files/tweets_lemm_test.csv')

test.head()

,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


# lemmatize

In [ ]:
from pymystem3 import Mystem
import re

In [8]:
corpus = train['text'].values.astype('U') # Переведём тексты в Unicode

corpus[0]

In [21]:
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)

    return lemm_text

In [22]:
lemmatize(corpus[0])

'@first_timee хоть я и школоть, но поверять, у мы то же самый :D общество профилирующий предмет тип)\n'

In [23]:
def clear_text(text):
    clean_text = re.sub(r'[^а-яА-ЯёЁ]',' ',text)
    clean_text = " ".join(clean_text.split())

    return(clean_text)

In [24]:
lemmatize(clear_text(corpus[0]))

'хоть я и школоть но поверять у мы то же самый общество профилировать предмет тип\n'

# bag of words, stop words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [26]:
count_vect = CountVectorizer()
bow = count_vect.fit_transform(corpus)

print("Размер мешка без учёта стоп-слов:", bow.shape)

Размер мешка без учёта стоп-слов: (5000, 18697)


In [27]:
from nltk.corpus import stopwords as nltk_stopwords

In [28]:
stop_words = nltk_stopwords.words('russian')

count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(corpus)

print("Размер мешка с учётом стоп-слов:", bow.shape)

Размер мешка с учётом стоп-слов: (5000, 18555)


N-граммы (по N **слов**, не символов):

In [29]:
count_vect = CountVectorizer(ngram_range=(2, 2))
n_gramm = count_vect.fit_transform(corpus)

print("Размер:", n_gramm.shape)

Размер: (5000, 41914)


# TF-IDF

Оценка важности слова определяется величиной $TF-IDF$ (от англ. _term frequency_, «частота терма, или слова»; _inverse document frequency_, «обратная частота документа, или текста»).  

То есть $TF$ отвечает за количество упоминаний слова в отдельном тексте, а $IDF$ отражает частоту его употребления во всём корпусе.  

$$TF-IDF = TF * IDF$$  

$TF = \frac{t}{n}$, где $t$ (от англ. _term_) — количество употребления слова; $n$ — общее число слов в тексте.  
$IDF = \log{\frac{D}{d}}$, где $D$ - общее число текстов в корпусе; d - количество текстов, в которых данное слово встречается.

$IDF$ нужна в формуле, чтобы уменьшить вес слов, наиболее распространённых в любом другом тексте заданного корпуса.

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords as nltk_stopwords

In [ ]:
stop_words = nltk_stopwords.words('russian')

In [31]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_idf = count_tf_idf.fit_transform(corpus)

print("Размер матрицы:", tf_idf.shape)

Размер матрицы: (5000, 18555)


> Передав `TfidfVectorizer()` аргумент `ngram_range`, можно рассчитать N-граммы.

# sentiment analysis

Анализ тональности текста, или сентимент-анализ (от англ. sentiment, «настроение»), выявляет эмоционально окрашенные слова. Этот инструмент помогает компаниям оценивать, например, реакцию на запуск нового продукта в интернете. На разбор тысячи отзывов человек потратит несколько часов, а компьютер — пару минут.  

Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. 

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords as nltk_stopwords

In [54]:
stop_words = nltk_stopwords.words('russian')

tf_idf_transformer = TfidfVectorizer(stop_words=stop_words)

tf_idf_transformer = tf_idf_transformer.fit(train['lemm_text'])

In [55]:
X_train = tf_idf_transformer.transform(train['lemm_text'])
X_test = tf_idf_transformer.transform(test['lemm_text'])

y_train = train['positive']

In [56]:
model = LogisticRegression()
model = model.fit(X_train, y_train)

In [57]:
test['positive'] = model.predict(X_test)
 
test.to_csv('./files/predictions', index=False)

test.head()

,text,lemm_text,positive
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...,1
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...,0
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...,0
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...,0
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...,1


In [59]:
# check
X_train_train, X_train_valid, y_train_train, y_train_valid = train_test_split(
    X_train, 
    y_train, 
    test_size=0.25, 
    random_state=42
)


model = LogisticRegression()
model = model.fit(X_train_train, y_train_train)

y_pred = model.predict(X_train_valid)

metrics.roc_auc_score(y_train_valid, y_pred)

0.6231414352420743

# Word2vec

Рассмотрим популярный метод построения языковых представлений — word2vec (от англ. word to vector, «слова к вектору»).  

Упрощённо разберём, как работает word2vec. Из примера с амадиной и тупиком вы узнали, что сходство этих слов определяется соседством с «красным клювом». А «муравьед» и «ленивец» часто встречаются в предложениях о Перу. То есть смысл слов определяется их контекстом.  

Тогда задача word2vec — предсказать: соседи или нет — заданные слова. Слова считаются соседями, если находятся в одном «окне» (максимальном расстоянии между словами). Выходит, пара слов — это признаки, а являются ли они соседями — целевой признак.

https://habr.com/ru/post/446530/ - Word2vec в картинках

# BERT

**BERT** (от англ. _Bidirectional Encoder Representations from Transformers_, «двунаправленная нейронная сеть-кодировщик») — нейронная сеть для создания модели языка. Её разработали в компании Google, чтобы повысить релевантность результатов поиска. Этот алгоритм понимает контекст запросов, а не просто анализирует фразы. Для машинного обучения она ценна тем, что помогает строить векторные представления. Причём в анализе текстов применяют уже предобученную на большом корпусе модель. Такие предобученные версии **BERT** годятся для работы с текстами на 104 языках мира, включая русский.  

**BERT** — это результат эволюции модели **word2vec**. В ходе её развития были придуманы и другие модели: **FastText** (англ. «быстрый текст»), **GloVe** (англ. _Global Vectors for Word Representation_, «глобальные векторы для языкового представления»), **ELMO** (англ. _Embeddings from Language Models_, «вложения языковых моделей») и **GPT** (англ. _Generative Pre-Training Transformer_, «предобученный трансформер для генерации»). Сейчас самые точные — это **BERT** и **GPT-3**, которого нет в открытом доступе.  

**BERT** учитывает контекст не только соседних слов, но и более дальних родственников. Работает так:
На входе модель получает, например, такую фразу: `«Красный клюв тупика [MASK] на голубом [MASK]»`, где MASK (англ. «маска») — это неизвестные слова, будто закрытые маской. Модель должна угадать эти спрятанные слова.
Модель обучается определять, связаны ли в предложении слова между собой. У нас были скрыты такие слова: «мелькнул» и «небе». Модель должна понять, что одно слово — продолжение другого. Скажем, если вместо «мелькнул» спрятать слово «прополз», то связи модель не найдёт.  

Перед вами большой датасет с твитами. Нужно научиться определять, какие твиты негативной тональности, а какие — позитивной. Чтобы решить эту задачу, из открытого репозитория [DeepPavlov](http://docs.deeppavlov.ai/en/master/features/models/bert.html) возьмём модель **RuBERT**, обученную на разговорном русскоязычном корпусе.  

Решим эту задачу на **PyTorch** (англ. «факел для Python»). Глубоко разбираться в средствах этой библиотеки мы не будем. Она применяется в задачах обработки естественного текста и компьютерного зрения. А нам нужна для работы с моделями типа BERT. Они находятся в библиотеке _transformers_ (англ. «трансформеры»): https://huggingface.co/transformers/.  

Возьмем всякие заготовки. Пошел [сюда](http://docs.deeppavlov.ai/en/master/features/models/bert.html), и скачал две версии (в формате `deeppavlov_pytorch`): 
* **RuBERT** was trained on the Russian part of Wikipedia and news data. We used this training data to build vocabulary of Russian subtokens and took multilingual version of BERT-base as initialization for RuBERT 1.  
* **Conversational RuBERT** was trained on OpenSubtitles 5, Dirty, Pikabu, and Social Media segment of Taiga corpus 8. We assembled new vocabulary for Conversational RuBERT model on this data and initialized model with RuBERT.  

Сохранил в `./files/rubert/` и `./files/ru_conversational/` соответственно. Там всё:  
* `vocab.txt`
* `bert_config.json`
* `pytorch_model.bin`

Погнали!

In [60]:
import torch
import transformers
from tqdm.notebook import tqdm

In [61]:
path_prefix = './files/ru_conversational/' # './files/rubert/' './files/ru_conversational/'

tokenizer = transformers.BertTokenizer(vocab_file=path_prefix+'vocab.txt')
config = transformers.BertConfig.from_json_file(path_prefix+'bert_config.json')
model = transformers.BertModel.from_pretrained(path_prefix+'pytorch_model.bin', config=config)

Some weights of the model checkpoint at ./files/ru_conversational/pytorch_model.bin were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Начнем с токенизации:

In [62]:
tokenizer.encode('Очень удобно использовать уже готовый трансформатор текста', add_special_tokens=True)

[101, 1094, 4980, 3373, 1034, 6037, 323, 73634, 10316, 102]

Для корректной работы модели мы указали аргумент `add_special_tokens` (англ. «добавить специальные токены»), равный `True`. Это значит, что к любому преобразуемому тексту добавляется токен начала (101) и токен конца текста (102).  

In [63]:
# чтобы долго не обрабатывать - возьмем маленький сэмпл
df_tweets = train[['text', 'positive']].sample(400).reset_index(drop=True)

df_tweets.head()

,text,positive
0,@bitchpleasemary верхнее эт ад((((((((\r\nя ры...,0
1,"кстати. я вчера узнала, что настоящий 3д принт...",1
2,посмотрела сопливую мелодраму и влюбилась в г...,0
3,@Anuta1961 ахахаха Огооонь:))) как там твоя ма...,1
4,@_13417 ахаха. Хорошо. А я ток домой. С 8свали...,1


In [66]:
tokenized = df_tweets['text'].apply( # BERT'у не нужна лемматизация!
    lambda x: tokenizer.encode(x, add_special_tokens=True)
)

tokenized.head()

0    [101, 168, 18196, 2537, 22750, 17408, 15190, 2...
1    [101, 2451, 132, 358, 4806, 19914, 128, 825, 2...
2    [101, 29968, 25733, 2111, 57255, 878, 322, 580...
3    [101, 168, 5335, 2598, 76511, 20924, 139, 6062...
4    [101, 168, 230, 19058, 4545, 34843, 132, 1643,...
Name: text, dtype: object

In [67]:
# максимальная возможная длина твита в токенах
max_len = tokenized.apply(lambda x: len(x)).max()

max_len

86

In [68]:
# заполняем нулями до максимальной длины, чтоб одна форма была
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

padded.shape

(400, 86)

In [70]:
# ставим attention=1 только там, где реальный токен, а не заполняющий дырки ноль
attention_mask = np.where(padded != 0, 1, 0)

attention_mask.shape

(400, 86)

А теперь к собственно эмбеддингам:

In [71]:
batch_size = 100 # по батчам будем обрабатывать

embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
    # преобразуем данные в torch Tensor
    batch = torch.LongTensor(
        padded[batch_size*i:batch_size*(i+1)]
    )
    # преобразуем маску в torch Tensor
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad(): # так быстрее
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].numpy()) # см. пояснения ниже
    
    # verbose
    print(i, pd.to_datetime('now').time().isoformat())
    
features = np.concatenate(embeddings)

  0%|          | 0/4 [00:00<?, ?it/s]

0 05:18:11.197495
1 05:18:20.525155
2 05:18:30.582974
3 05:18:39.419351


`batch_embeddings[0][:,0,:]`: почему так?  

в `batch_embeddings` - два тензора. Первый - это *last_hidden_state*, значения эмбеддингов для каждого токена каждого твита из батча. Второй - это *pooler_output*, специально обработанный выход для первого токена (того самого, который и мы берем в итоге): + полносвязный линейный слой и тангенс-активация. Полное описание того, что возвращает модель, есть [тут](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward.returns). В общем, берем первый, *last_hidden_state*.

`[:,0,:]` - это срез (slice) трёхмерного массива. Эмбеддинг превращает каждое слово твита (токен) в вектор длины 768. То есть в нашем батче из 100 твитов, каждый ширины 133 (т.к. мы заполнили все, что было короче 133 токенов, нулями, в padded) появляется третье измерение: размерность эмбеддинга, "глубина". То есть по сути, для каждого твита мы берем эмбеддинг только первого слова (все 100 твитов батча, все 768 значений вектора эмбеддинга, но только 0-е слово).  

На самом деле это не первое слово твита, т.к. при вызове `tokenizer.encode(x, add_special_tokens=True)` мы добавляем перед токенами самого твита специальный токен `[CLF]`. Это специальный токен, собирающий всю необходимую информацию для классификации. Его эмбеддинг мы и берем, и его же эмбеддинг (немного доработанный) возвращается в *pooler_output*, втором тензоре.  

В общем, запомнить нужно следующее:
* первый токен в токенизированном твите (после `tokenizer.encode()` если `add_special_tokens=True`) - это специальный токен `[CLF]`, чей эмбеддинг по-сути является эмбеддингом целого предложения для задачи классификации.
* модель возвращает эмбеддинги для каждого токена (*last_hidden_state*), а также специально дообработанный эмбеддинг специального токена `[CLF]` (*pooler_output*). Т.е. два тензора, первый трёхмерный, второй - двумерный (*last_hidden_state* и *pooler_output*).
* для дальнейшей классификации в качестве эмбеддинга всего твита можно использовать `last_hidden_state[:,0,:]` или `pooler_output`. Там примерно одно и то же, только в `pooler_output` эмбеддинг  `[CLF]` (`last_hidden_state[:,0,:]`) немного дообработан (+ полносвязный линейный слой и тангенс-активация)

In [73]:
# check
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    df_tweets['positive'], 
    test_size=0.5, 
    random_state=42
)

model = LogisticRegression()
model = model.fit(X_train, y_train)

y_pred = model.predict(X_test)

metrics.roc_auc_score(y_test, y_pred)

0.9189244663382594

https://stepik.org/course/54098/syllabus  
https://github.com/Samsung-IT-Academy/stepik-dl-nlp  
https://github.com/Samsung-IT-Academy/stepik-dl-nlp/blob/master/task2_word_embeddings.ipynb  
https://github.com/Samsung-IT-Academy/stepik-dl-nlp/blob/master/task9_bert_sentiment_analysis.ipynb  

https://rusvectores.org/ru/models/  
geowac_tokens_none_fasttextskipgram_300_5_2020  
https://fasttext.cc/  
https://github.com/facebookresearch/fastText  


In [7]:
lol = np.load(
    file='./files/214/model.model.vectors_ngrams.npy', 
    mmap_mode=None, 
    allow_pickle=False, 
    fix_imports=True, 
    encoding='ASCII'
)

lol[:5, :5]

array([[-1.18202202e-04,  2.35366239e-03,  1.78556633e-03,
        -1.04790344e-03, -1.13029417e-03],
       [ 3.35591048e-01, -1.10159373e+00, -1.14366546e-01,
        -6.55330002e-01, -1.21734643e+00],
       [-5.56692481e-04, -2.73203687e-03,  8.11694845e-05,
        -4.20838507e-04, -2.95227626e-03],
       [-2.45295785e-04,  1.30942592e-03, -1.32767402e-03,
         3.04132444e-03, -2.79921456e-03],
       [-1.08011405e-03,  3.30130872e-03, -2.84952554e-03,
         1.47060910e-03,  7.80855771e-04]], dtype=float32)

In [8]:
lol[:5, -5:]

array([[-2.0940178e-03,  3.2052917e-03,  3.1521744e-03,  2.2887776e-03,
        -2.8068325e-03],
       [ 4.0677685e-01, -2.2524649e-01,  4.4718465e-01, -1.7501085e-01,
         4.2675009e-01],
       [-2.5399609e-03, -3.2787060e-03,  3.1196531e-03,  3.5778832e-04,
         2.5944707e-03],
       [-7.4223144e-04,  6.9249352e-04,  1.5482459e-03, -1.0184679e-03,
        -6.1467348e-04],
       [ 2.3629216e-03, -1.0944166e-03, -1.4347958e-03,  2.7006823e-03,
         2.0544571e-03]], dtype=float32)

# natasha navec

https://github.com/natasha/navec  

Скачал `navec_hudlit_v1_12B_500K_300d_100q.tar`, положил в `./files`. Попробуем:

In [9]:
# !pip install navec

In [10]:
from navec import Navec

In [11]:
navec = Navec.load(
    './files/navec_hudlit_v1_12B_500K_300d_100q.tar'
)

In [12]:
# embedding
navec['навек']

array([ 0.3955571 ,  0.11600914,  0.24605067, -0.35206917, -0.08932345,
        0.3382279 , -0.5457616 ,  0.07472657, -0.4753835 , -0.3330848 ,
        0.1449912 , -0.13690177,  0.0840969 , -0.2141802 , -0.57312167,
       -0.06113251,  0.04558022, -0.30147526,  0.55966735, -0.30724582,
       -0.23867525, -0.28796813, -0.10781458,  0.09957517,  0.25810853,
        0.05649512, -0.03052731,  0.06966446,  0.01507293, -0.14274776,
       -0.40885502,  0.2893703 ,  0.3201632 , -0.27072856, -0.04702847,
        0.41056094,  0.51552117, -0.27452162, -0.10154261,  0.41732824,
        0.15827584, -0.03746929,  0.17745009, -0.03559239, -0.29721352,
       -0.39473224, -0.04143994,  0.05495927,  0.47845373, -0.32670122,
        0.14475909,  0.35896015, -0.3800541 ,  0.29924598, -0.31741822,
       -0.71888554, -0.35691768, -0.2958643 , -0.37184098,  0.08903536,
       -0.12709911, -0.06632375,  0.03680592,  0.27049237,  0.00893382,
       -0.20365159, -0.27380955,  0.08020782,  0.12610178,  0.04

In [13]:
# check
'нааавееек' in navec

False

In [14]:
navec.get('нааавееек') # None

In [15]:
# index of word
navec.vocab['навек']

225823

In [16]:
# with check
navec.vocab.get('наааавеeeк', navec.vocab.unk_id)

500000

In [17]:
# special word 
navec['<unk>']

array([ 2.14312136e-01,  3.70287180e-01,  1.36796311e-01, -1.86535835e-01,
       -4.91572991e-02, -1.88751370e-02, -1.32190725e-02,  3.57939489e-02,
       -3.97918969e-02,  1.94128811e-01, -8.25866908e-02,  2.47690390e-04,
        2.88263001e-02,  2.01173663e-01, -1.54272586e-01, -1.13935187e-01,
       -4.74858470e-02, -7.92295299e-03,  6.50205165e-02, -3.78477909e-02,
        6.24999329e-02,  2.49566153e-01,  1.03294946e-01, -2.11493388e-01,
       -1.73085947e-02, -2.82477215e-02, -6.87575415e-02, -9.21097770e-02,
        8.71437322e-03, -1.69095173e-01, -5.73454238e-02,  4.21022065e-02,
       -5.24346411e-01, -1.58332035e-01,  5.83604947e-02, -6.78519439e-03,
       -6.93208054e-02, -5.74708311e-03, -1.20353170e-01, -4.40001450e-02,
        4.75032702e-02, -2.33378902e-01, -1.33015737e-01,  1.27385199e-01,
       -7.16302916e-02,  1.28748834e-01,  1.13330811e-01,  1.26265138e-02,
        5.89972734e-02,  2.43283421e-01, -8.16499963e-02,  2.72306442e-01,
        1.67372063e-01, -

In [18]:
# special word 
navec['<pad>']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# fasttext

https://fasttext.cc/docs/en/crawl-vectors.html  - Russian, bin
https://radimrehurek.com/gensim/models/fasttext.html  



In [ ]:
from gensim.test.utils import datapath
from gensim.models.fasttext import load_facebook_vectors

In [ ]:
cap_path = datapath("crime-and-punishment.bin")
wv = load_facebook_vectors(cap_path)

In [ ]:
'landlord' in wv.key_to_index  # Word is out of vocabulary



True


In [ ]:
oov_vector = wv['landlord']  # Even OOV words have vectors in FastText

In [ ]:
'landlady' in wv.key_to_index  # Word is in the vocabulary

In [ ]:
iv_vector = wv['landlady']

# RusVectores

https://rusvectores.org/ru/models/  
https://habr.com/ru/post/275913/  


In [19]:
import pymorphy2

In [20]:
def canonize_words(words: list) -> list:
    stop_words = ('быть', 'мой', 'наш', 'ваш', 'их', 'его', 'её', 'их',
                  'этот', 'тот', 'где', 'который', 'либо', 'нибудь', 'нет', 'да')
    grammars = {'NOUN': '_S',
                'VERB': '_V', 'INFN': '_V', 'GRND': '_V', 'PRTF': '_V', 'PRTS': '_V',
                'ADJF': '_A', 'ADJS': '_A',
                'ADVB': '_ADV',
                'PRED': '_PRAEDIC'}

    morph = pymorphy2.MorphAnalyzer()
    normalized = []
    for i in words:
        forms = morph.parse(i)
        try:
            form = max(forms, key=lambda x: (x.score, x.methods_stack[0][2]))
        except Exception:
            form = forms[0]
            print(form)
        if not (form.tag.POS in ['PREP', 'CONJ', 'PRCL', 'NPRO', 'NUMR']
                or 'Name' in form.tag
                or 'UNKN' in form.tag
                or form.normal_form in stop_words):  # 'ADJF'
            normalized.append(form.normal_form + grammars.get(form.tag.POS, ''))
    return normalized

In [21]:
canonize_words(['лол', 'кек'])

['кек_S']

In [ ]:
sem.load_w2v_model(sem.WORD2VEC_MODEL_FILE)

In [ ]:
def make_data_model(file_name: str) -> dict:
    poems = read_poems(file_name)
    bags, voc = make_bags(poems)
    w2v_model = sem.load_w2v_model(sem.WORD2VEC_MODEL_FILE)
    sd = [sem.semantic_density(bag, w2v_model, unknown_coef=-0.001) for bag in bags]
    sa = [sem.semantic_association(bag, w2v_model) for bag in bags]
    rates = [0.0 for _ in range(len(poems))]
    return {'poems'       : poems,
            'bags'        : bags,
            'vocabulary'  : voc,
            'density'     : sd,
            'associations': sa,
            'rates'       : rates}

In [ ]:
def semantic_similarity(bag1, bag2: list, w2v_model, unknown_coef=0.0) -> float:
    sim_sum = 0.0
    for i in range(len(bag1)):
        for j in range(len(bag2)):
            try:
                sim_sum += w2v_model.similarity(bag1[i], bag2[j])
            except Exception:
                sim_sum += unknown_coef
    return sim_sum / (len(bag1) * len(bag2))

In [ ]:
https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.load_facebook_vectors
    

In [ ]:
gensim.KeyedVectors.load_word2vec_format

In [ ]:
from gensim.test.utils import datapath
>>>
cap_path = datapath("crime-and-punishment.bin")
wv = load_facebook_vectors(cap_path)
>>>
'landlord' in wv.key_to_index  # Word is out of vocabulary
False
oov_vector = wv['landlord']  # Even OOV words have vectors in FastText
>>>
'landlady' in wv.key_to_index  # Word is in the vocabulary
True
iv_vector = wv['landlady']

In [ ]:
existent_word = "computer"
existent_word in model.wv.key_to_index
True
computer_vec = model.wv[existent_word]  # numpy vector of a word
>>>
oov_word = "graph-out-of-vocab"
oov_word in model.wv.key_to_index
False
oov_vec = model.wv[oov_word]  # numpy vector for OOV word

In [ ]:
cap_path = datapath("crime-and-punishment.bin")
fb_model = load_facebook_model(cap_path)